In [1]:
# // Dependencies
import pandas as pd
import matplotlib
matplotlib.use('TKAgg')
import matplotlib.pyplot as plt
import sys
import os
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from tkinter import filedialog
import glob

In [2]:
# // Path to Query
folder_path = 'QUERY_FILE_GOES_HERE/'

xlsx_files = glob.glob(os.path.join(folder_path, '*.xlsx'))

QUERYS = {}
Q_Index = 0

for file_path in xlsx_files:
    OPName = os.path.splitext(os.path.basename(file_path))[0] # Extract filename without extension as DataFrame name
    QUERYS[OPName] = pd.read_excel(file_path)
    Q_Index += 1
    
print(f"Files in folder '{folder_path}':")

if Q_Index > 0:
    for file_name in QUERYS.keys():
        print(file_name)
else:
   print(f"No .xlsx file found in {folder_path}")

Files in folder 'QUERY_FILE_GOES_HERE/':
Osceola prosper query 01-02-2023 Nelson-1
Osceola prosper query 01-19-2023 Nelson-1


In [ ]:
root = tk.Tk()
root.withdraw()
Query = filedialog.askopenfilename(initialdir='QUERY_FILE_GOES_HERE/', title='Select a file', filetypes=[('Excel files', '*.xlsx')])

root = tk.Tk()
root.title("Current Term Input")

label = tk.Label(root, text="Input current term in Banner format ######. Ex. 202210: ")
label.pack()

entry = tk.Entry(root)
entry.pack()

def get_term():
    global Current_Term
    global LastTwo
    Current_Term = entry.get()
    if Current_Term.isdigit():
        Current_Term = int(Current_Term)
        LastTwo = Current_Term % 100
        if Current_Term > 100000 and Current_Term < 999999 and (LastTwo == 10 or LastTwo == 20 or LastTwo == 30):
            root.destroy()
        else:
            messagebox.showerror("Error", "Input not in correct Banner format")
    else:
        messagebox.showerror("Error", "Input is not a number")

submit_btn = tk.Button(root, text="Submit", command=get_term)
submit_btn.pack()

root.mainloop()

In [ ]:
# Returns Count of VIDs as Population
TPS = Query["VID"].count()

# Sets Term Codes & Reads Current Term from input
if LastTwo == 10:
    Fall = Current_Term
    Term_Actual = "Fall"
elif LastTwo == 20:
    Fall = Current_Term - 10
    Term_Actual = "Spring"
elif LastTwo == 30:
    Fall = Current_Term - 20
    Term_Actual = "Summer"
    Spring = Fall + 10
    Summer = Spring + 10

# Returns Count of Enrolled OP Students in Current Term
CTerm_Search = str(Current_Term) + "_Enroll_Status"
CTerm = Query[CTerm_Search].count()

# Returns Count of Enrolled OP Students in NEXT Term
# Sets Next Term Variable
if LastTwo == 30:
    NTerm_Search = str(Spring) + "_Enroll_Status"
elif LastTwo == 10 or LastTwo == 20:
    NTerm_Search = str(Current_Term + 10) + "_Enroll_Status"
NTerm = Query[NTerm_Search].count()

# 1. Filters out OP Students NOT ENROLLED in current Term
# 2. Returns count of Current Term Enrollees that Enrolled in the Following Term
# 3. Returns count of Current Term withdrawls
Current_E_List = Query[Query[CTerm_Search].notnull()]
Next_E_List_Count = Current_E_List[NTerm_Search].count()
Current_WD = len(Current_E_List[Current_E_List[CTerm_Search] == "WT"])

# 1. Filters OP Students Admitted Next Term or later
# 2. Out of filtered OP Students, counts enrolled students in the next term
DA_NTerm_List = Query[Query["TermCodeAdmit"] > Current_Term]
Admit_DA_NTerm = DA_NTerm_List[NTerm_Search].count()

# Index for graphs: count of all students admitted during or after next term
DASIndex = len(Query[Query["TermCodeAdmit"] > Current_Term].count())